In [1]:
import requests
import pandas as pd
import datetime
Today = datetime.date.today()
Today_str = Today.strftime("%Y-%m-%d")

In [2]:
datetime.datetime.now().year

2024

In [3]:
Current_year = datetime.datetime.now().year
years = list(range(Current_year, Current_year -10, -1))
all_matches = []

In [4]:
type(Current_year)

int

In [5]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the URL
url = "https://fbref.com/en/comps/20/2023-2024/2023-2024-Bundesliga-Stats"
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all h1 tags
h1_tags = soup.find_all("h1")

# Iterate through the h1 tags to find the desired line
for h1_tag in h1_tags:
    if "Bundesliga Stats" in h1_tag.text:
        line = h1_tag.text.strip()
        break

# Print the result
print(line)


2023-2024 Bundesliga Stats


In [6]:
year = line.split("-")[1].split()[0]
year

'2024'

In [7]:
import re
year = re.search('\d+-(\d+)\s.+', line).group(1)
year

'2024'

In [8]:
import time
standings_url="https://fbref.com/en/comps/20/2023-2024/2023-2024-Bundesliga-Stats"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
    
previous_season = soup.select("a.prev")[0].get("href")
standings_url = f"https://fbref.com{previous_season}"
    
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    Scores_Fixtures = pd.read_html(data.text, match="Scores & Fixtures")[0]
    Scores_Fixtures = Scores_Fixtures[['Date','Time','Comp','Round','Day','Venue','GF','GA','Opponent','Poss']]

    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
        
    soup = BeautifulSoup(data.text)
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'all_comps/keeper/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
    goalkeeping.columns = goalkeeping.columns.droplevel()
        
    #try:
    team_data = Scores_Fixtures.merge(shooting[['Date','Sh']], on = ['Date'])
    team_data = team_data.merge(goalkeeping[['Date','Save%']], on = ['Date'])

    #except ValueError:
    #continue
    team_data = team_data[team_data["Comp"] == "Bundesliga"]
        
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(20)

C:\Users\tramn\AppData\Local\Temp\ipykernel_5756\357820671.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  Scores_Fixtures = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\tramn\AppData\Local\Temp\ipykernel_5756\357820671.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\tramn\AppData\Local\Temp\ipykernel_5756\357820671.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goalkeeping = pd.read_html(data.text, match="Goalkeeping")[0]
C:\Users\tramn\AppData\Local\Temp\ipykernel_5756\357820671.py:17: FutureWarning: Passing literal ht

In [9]:
len(all_matches)

18

In [10]:
all_matches

[          Date   Time        Comp         Round  Day Venue  GF  GA  \
 1   2023-08-19  15:30  Bundesliga   Matchweek 1  Sat  Home   3   2   
 2   2023-08-26  18:30  Bundesliga   Matchweek 2  Sat  Away   3   0   
 3   2023-09-02  15:30  Bundesliga   Matchweek 3  Sat  Home   5   1   
 4   2023-09-15  20:30  Bundesliga   Matchweek 4  Fri  Away   2   2   
 6   2023-09-24  15:30  Bundesliga   Matchweek 5  Sun  Home   4   1   
 7   2023-09-30  15:30  Bundesliga   Matchweek 6  Sat  Away   3   0   
 9   2023-10-08  15:30  Bundesliga   Matchweek 7  Sun  Home   3   0   
 10  2023-10-21  15:30  Bundesliga   Matchweek 8  Sat  Away   2   1   
 12  2023-10-29  17:30  Bundesliga   Matchweek 9  Sun  Home   2   1   
 14  2023-11-04  15:30  Bundesliga  Matchweek 10  Sat  Away   3   2   
 16  2023-11-12  15:30  Bundesliga  Matchweek 11  Sun  Home   4   0   
 17  2023-11-25  15:30  Bundesliga  Matchweek 12  Sat  Away   3   0   
 19  2023-12-03  17:30  Bundesliga  Matchweek 13  Sun  Home   1   1   
 21  2

In [11]:
match_df = pd.concat(all_matches)

In [12]:
match_df.columns = [c.lower() for c in match_df.columns]

In [13]:
import numpy as np
match_df[['gf', 'ga','poss','sh','save%']] = match_df[['gf', 'ga','poss','sh','save%']].astype(np.float64)
match_df['season'] = match_df['season'].astype(np.int64)

In [14]:
match_df

,date,time,comp,round,day,venue,gf,ga,opponent,poss,sh,save%,season,team
1,2023-08-19,15:30,Bundesliga,Matchweek 1,Sat,Home,3.0,2.0,RB Leipzig,44.0,11.0,66.7,2024,Bayer Leverkusen
2,2023-08-26,18:30,Bundesliga,Matchweek 2,Sat,Away,3.0,0.0,Gladbach,60.0,24.0,100.0,2024,Bayer Leverkusen
3,2023-09-02,15:30,Bundesliga,Matchweek 3,Sat,Home,5.0,1.0,Darmstadt 98,72.0,25.0,0.0,2024,Bayer Leverkusen
4,2023-09-15,20:30,Bundesliga,Matchweek 4,Fri,Away,2.0,2.0,Bayern Munich,49.0,12.0,77.8,2024,Bayer Leverkusen
6,2023-09-24,15:30,Bundesliga,Matchweek 5,Sun,Home,4.0,1.0,Heidenheim,71.0,20.0,50.0,2024,Bayer Leverkusen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2024-04-20,15:30,Bundesliga,Matchweek 30,Sat,Away,2.0,0.0,Köln,42.0,12.0,100.0,2024,Darmstadt 98
31,2024-04-28,19:30,Bundesliga,Matchweek 31,Sun,Home,0.0,1.0,Heidenheim,45.0,11.0,80.0,2024,Darmstadt 98
32,2024-05-04,15:30,Bundesliga,Matchweek 32,Sat,Away,0.0,3.0,Wolfsburg,47.0,6.0,57.1,2024,Darmstadt 98
33,2024-05-12,15:30,Bundesliga,Matchweek 33,Sun,Home,0.0,6.0,Hoffenheim,41.0,10.0,14.3,2024,Darmstadt 98


In [15]:
from google.cloud import storage
client = storage.Client()
bucket = client.get_bucket('bundesliga_0410')
blob = bucket.blob('matches.csv')
blob.download_to_filename('matches.csv')
df = pd.read_csv('matches.csv')

In [16]:
df = df.drop('index', axis =1)
df

,date,time,comp,round,day,venue,gf,ga,opponent,poss,sh,save%,season,team
0,2023-08-19,15:30,Bundesliga,Matchweek 1,Sat,Home,3.0,2.0,RB Leipzig,44.0,11.0,66.7,2024,Bayer Leverkusen
1,2023-08-26,18:30,Bundesliga,Matchweek 2,Sat,Away,3.0,0.0,M'Gladbach,60.0,24.0,100.0,2024,Bayer Leverkusen
2,2023-09-02,15:30,Bundesliga,Matchweek 3,Sat,Home,5.0,1.0,Darmstadt 98,72.0,25.0,0.0,2024,Bayer Leverkusen
3,2023-09-15,20:30,Bundesliga,Matchweek 4,Fri,Away,2.0,2.0,Bayern Munich,49.0,12.0,77.8,2024,Bayer Leverkusen
4,2023-09-24,15:30,Bundesliga,Matchweek 5,Sun,Home,4.0,1.0,Heidenheim,71.0,20.0,50.0,2024,Bayer Leverkusen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6726,2024-04-20,15:30,Bundesliga,Matchweek 30,Sat,Away,2.0,0.0,Köln,42.0,12.0,100.0,2024,Darmstadt 98
6727,2024-04-28,19:30,Bundesliga,Matchweek 31,Sun,Home,0.0,1.0,Heidenheim,45.0,11.0,80.0,2024,Darmstadt 98
6728,2024-05-04,15:30,Bundesliga,Matchweek 32,Sat,Away,0.0,3.0,Wolfsburg,47.0,6.0,57.1,2024,Darmstadt 98
6729,2024-05-12,15:30,Bundesliga,Matchweek 33,Sun,Home,0.0,6.0,Hoffenheim,41.0,10.0,14.3,2024,Darmstadt 98


In [17]:
# union them
df_combined =pd.concat([df,match_df]).drop_duplicates().reset_index()
df_combined

,index,date,time,comp,round,day,venue,gf,ga,opponent,poss,sh,save%,season,team
0,0,2023-08-19,15:30,Bundesliga,Matchweek 1,Sat,Home,3.0,2.0,RB Leipzig,44.0,11.0,66.7,2024,Bayer Leverkusen
1,1,2023-08-26,18:30,Bundesliga,Matchweek 2,Sat,Away,3.0,0.0,M'Gladbach,60.0,24.0,100.0,2024,Bayer Leverkusen
2,2,2023-09-02,15:30,Bundesliga,Matchweek 3,Sat,Home,5.0,1.0,Darmstadt 98,72.0,25.0,0.0,2024,Bayer Leverkusen
3,3,2023-09-15,20:30,Bundesliga,Matchweek 4,Fri,Away,2.0,2.0,Bayern Munich,49.0,12.0,77.8,2024,Bayer Leverkusen
4,4,2023-09-24,15:30,Bundesliga,Matchweek 5,Sun,Home,4.0,1.0,Heidenheim,71.0,20.0,50.0,2024,Bayer Leverkusen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6760,24,2024-03-02,15:30,Bundesliga,Matchweek 24,Sat,Home,1.0,4.0,RB Leipzig,44.0,14.0,0.0,2024,Bochum
6761,8,2023-10-22,15:30,Bundesliga,Matchweek 8,Sun,Home,3.0,1.0,Gladbach,60.0,21.0,50.0,2024,Koln
6762,4,2023-09-17,17:30,Bundesliga,Matchweek 4,Sun,Home,3.0,3.0,Gladbach,37.0,15.0,70.0,2024,Darmstadt 98
6763,21,2024-02-10,15:30,Bundesliga,Matchweek 21,Sat,Away,0.0,0.0,Gladbach,42.0,10.0,100.0,2024,Darmstadt 98


In [18]:
updated_df= df_combined.to_csv(index=False)
filename_in_bucket = bucket.blob('matches.csv')
filename_in_bucket.upload_from_string(updated_df)

In [19]:
df[df['season'].astype(str) == "2024"].groupby(["round"]).count()

,date,time,comp,day,venue,gf,ga,opponent,poss,sh,save%,season,team
round,,,,,,,,,,,,,
Matchweek 1,18,18,18,18,18,18,18,18,18,18,18,18,18
Matchweek 10,18,18,18,18,18,18,18,18,18,18,18,18,18
Matchweek 11,18,18,18,18,18,18,18,18,18,18,18,18,18
Matchweek 12,18,18,18,18,18,18,18,18,18,18,18,18,18
Matchweek 13,18,18,18,18,18,18,18,18,18,18,18,18,18
Matchweek 14,18,18,18,18,18,18,18,18,18,18,18,18,18
Matchweek 15,18,18,18,18,18,18,18,18,18,18,18,18,18
Matchweek 16,18,18,18,18,18,18,18,18,18,18,18,18,18
Matchweek 17,18,18,18,18,18,18,18,18,18,18,17,18,18


In [20]:
df[(df["season"].astype(str) == "2024") & (df["round"].astype(str) == "Matchweek 32")]

,date,time,comp,round,day,venue,gf,ga,opponent,poss,sh,save%,season,team
6608,2024-05-03,20:30,Bundesliga,Matchweek 32,Fri,Away,1.0,1.0,Hoffenheim,57.0,5.0,100.0,2024,RB Leipzig
6609,2024-05-03,20:30,Bundesliga,Matchweek 32,Fri,Home,1.0,1.0,RB Leipzig,43.0,6.0,50.0,2024,Hoffenheim
6616,2024-05-05,17:30,Bundesliga,Matchweek 32,Sun,Away,5.0,1.0,Eint Frankfurt,45.0,7.0,80.0,2024,Bayer Leverkusen
6623,2024-05-04,15:30,Bundesliga,Matchweek 32,Sat,Home,3.0,1.0,Bayern Munich,61.0,24.0,100.0,2024,Stuttgart
6629,2024-05-04,15:30,Bundesliga,Matchweek 32,Sat,Away,1.0,3.0,Stuttgart,39.0,9.0,66.7,2024,Bayern Munich
6636,2024-05-03,20:30,Bundesliga,Matchweek 32,Fri,Away,1.0,1.0,Hoffenheim,45.0,8.0,87.5,2024,RB Leipzig
6643,2024-05-04,15:30,Bundesliga,Matchweek 32,Sat,Home,5.0,1.0,Augsburg,61.0,21.0,66.7,2024,Dortmund
6648,2024-05-05,17:30,Bundesliga,Matchweek 32,Sun,Home,1.0,5.0,Leverkusen,55.0,17.0,50.0,2024,Eintracht Frankfurt
6654,2024-05-03,20:30,Bundesliga,Matchweek 32,Fri,Home,1.0,1.0,RB Leipzig,55.0,20.0,75.0,2024,Hoffenheim
6662,2024-05-05,19:30,Bundesliga,Matchweek 32,Sun,Home,1.0,1.0,Mainz 05,46.0,15.0,66.7,2024,Heidenheim


In [21]:
df

,date,time,comp,round,day,venue,gf,ga,opponent,poss,sh,save%,season,team
0,2023-08-19,15:30,Bundesliga,Matchweek 1,Sat,Home,3.0,2.0,RB Leipzig,44.0,11.0,66.7,2024,Bayer Leverkusen
1,2023-08-26,18:30,Bundesliga,Matchweek 2,Sat,Away,3.0,0.0,M'Gladbach,60.0,24.0,100.0,2024,Bayer Leverkusen
2,2023-09-02,15:30,Bundesliga,Matchweek 3,Sat,Home,5.0,1.0,Darmstadt 98,72.0,25.0,0.0,2024,Bayer Leverkusen
3,2023-09-15,20:30,Bundesliga,Matchweek 4,Fri,Away,2.0,2.0,Bayern Munich,49.0,12.0,77.8,2024,Bayer Leverkusen
4,2023-09-24,15:30,Bundesliga,Matchweek 5,Sun,Home,4.0,1.0,Heidenheim,71.0,20.0,50.0,2024,Bayer Leverkusen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6726,2024-04-20,15:30,Bundesliga,Matchweek 30,Sat,Away,2.0,0.0,Köln,42.0,12.0,100.0,2024,Darmstadt 98
6727,2024-04-28,19:30,Bundesliga,Matchweek 31,Sun,Home,0.0,1.0,Heidenheim,45.0,11.0,80.0,2024,Darmstadt 98
6728,2024-05-04,15:30,Bundesliga,Matchweek 32,Sat,Away,0.0,3.0,Wolfsburg,47.0,6.0,57.1,2024,Darmstadt 98
6729,2024-05-12,15:30,Bundesliga,Matchweek 33,Sun,Home,0.0,6.0,Hoffenheim,41.0,10.0,14.3,2024,Darmstadt 98
